In [1]:
import os
from ultralytics import YOLO
from glob import glob
from sklearn.model_selection import train_test_split
import yaml
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# 数据集路径 修改路径配置在自己的yolo_materials文件夹中
dataset_path = 'D:/AI50期直通车/yolo_materials/datasets/Fatigue/'

# 获取所有图像文件和标注文件路径
image_files = glob(os.path.join(dataset_path, 'images', '*.jpg'))
annotation_files = glob(os.path.join(dataset_path, 'Annotations', '*.xml'))

In [3]:
# 处理没有标签的图片，只要第一次运行
# for pic in image_files:
#     anno_name = (pic.split('.')[0] + '.xml').replace('images','Annotations')
#     if anno_name not in annotation_files:
#         os.remove(os.path.join(dataset_path,pic))

In [4]:
# 转换VOC格式数据为YOLO格式
import xml.etree.ElementTree as ET

classes = ['closed_eye', 'open_eye','closed_mouth','open_mouth']  # 根据你的数据集类别修改

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(image_id):
    in_file = open(dataset_path+'Annotations/%s.xml' % (image_id))
    out_file = open(dataset_path+'labels/%s.txt' % (image_id), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

if not os.path.exists(dataset_path+'labels/'):
    os.makedirs(dataset_path+'labels/')
for filename in annotation_files:
    if filename.endswith('.xml'):
        # file_path = os.path.join(path, filename)
        image_id = os.path.splitext(os.path.basename(filename))[0]
        convert_annotation(image_id)



In [5]:
# 进行8:2分割

labels_files = glob(os.path.join(dataset_path, 'labels', '*.txt'))
train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, labels_files, test_size=0.2, random_state=42)

In [6]:
# 将文件路径写入文本文件
if not os.path.exists(dataset_path+'paths/'):
    os.makedirs(dataset_path+'paths/')
def write_file_list(file_list, filename):
    filename = os.path.join(dataset_path,'paths', filename)
    with open(filename, 'w') as f:
        for file in file_list:
            f.write(f"{file}\n")

write_file_list(train_images, 'train_images.txt')
write_file_list(val_images, 'val_images.txt')
write_file_list(train_labels, 'train_labels.txt')
write_file_list(val_labels, 'val_labels.txt')

In [7]:
# 生成my_det_data.yaml文件
data = {
    'train': 'paths/train_images.txt',
    'val': 'paths/val_images.txt',
    'nc': 4,  
    'names': ['closed_eye', 'open_eye','closed_mouth','open_mouth']
}

with open('Fatigue_det_data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


In [8]:
# Load a model
model = YOLO("yolov8n.yaml") 

In [9]:
# Train the model
results = model.train(data="Fatigue_det_data.yaml", 
                      epochs=10, 
                      imgsz=320, 
                      batch=2,
                      workers=1)

Ultralytics YOLOv8.2.60  Python-3.11.7 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=Fatigue_det_data.yaml, epochs=10, time=None, patience=100, batch=2, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning D:\AI50期直通车\yolo_materials\datasets\Fatigue\labels... 2331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2331/2331 [00:02<00:00, 817.64it/s]


train: New cache created: D:\AI50\yolo_materials\datasets\Fatigue\labels.cache


val: Scanning D:\AI50期直通车\yolo_materials\datasets\Fatigue\labels... 583 images, 0 backgrounds, 0 corrupt: 100%|██████████| 583/583 [00:01<00:00, 582.41it/s]


val: New cache created: D:\AI50\yolo_materials\datasets\Fatigue\labels.cache
Plotting labels to D:\AI50\yolo_materials\ultralytics\runs\detect\train28\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to D:\AI50\yolo_materials\ultralytics\runs\detect\train28
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10         0G        1.8      1.186     0.9225          2        320: 100%|██████████| 1166/1166 [06:25<00:00,  3.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [00:35<00:00,  4.17it/s]

                   all        583       1572      0.909      0.871      0.948      0.489



10 epochs completed in 1.409 hours.
Optimizer stripped from D:\AI50\yolo_materials\ultralytics\runs\detect\train28\weights\last.pt, 6.2MB
Optimizer stripped from D:\AI50\yolo_materials\ultralytics\runs\detect\train28\weights\best.pt, 6.2MB

Validating D:\AI50\yolo_materials\ultralytics\runs\detect\train28\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.11.7 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i7-1260P)
YOLOv8n summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [00:32<00:00,  4.52it/s]


                   all        583       1572      0.909      0.871      0.948      0.489
            closed_eye        223        409      0.873      0.851      0.926      0.425
              open_eye        326        595      0.916      0.756      0.912      0.412
          closed_mouth        515        516      0.943      0.973      0.988      0.531
            open_mouth         52         52      0.904      0.905      0.967      0.588
Speed: 0.4ms preprocess, 48.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to D:\AI50\yolo_materials\ultralytics\runs\detect\train28
